# Part 2: Submission with Fine-Tuned LoRA Model

This notebook loads the base DeBERTa model and the fine-tuned LoRA adapters, calibrates the probabilities, and generates the final submission.

## 1. Install Libraries

In [12]:
#not needed for kaggle
#!pip install -q transformers peft accelerate bitsandbytes torch scikit-learn

## 2. Load Data and Model

In [13]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from peft import PeftModel

# Load data for validation set (for calibration) and test set
#path = '../dataset/'
path = '/kaggle/input/llm-classification-finetuning/'
train_df = pd.read_csv(path + 'train.csv')
test_df = pd.read_csv(path + 'test.csv')
submission_df = pd.read_csv(path + 'sample_submission.csv')

# --- Recreate the validation set to match the one used in training ---
conditions = [train_df['winner_model_a'] == 1, train_df['winner_model_b'] == 1, train_df['winner_tie'] == 1]
choices = [0, 1, 2]
train_df['label'] = np.select(conditions, choices, default=-1)
train_df = train_df[train_df['label'] != -1].copy()

def create_text(row):
    return f"""prompt: {row['prompt']}

response_a: {row['response_a']}

response_b: {row['response_b']}"""

train_df['text'] = train_df.apply(create_text, axis=1)
test_df['text'] = test_df.apply(create_text, axis=1)

_, val_texts, _, val_labels = train_test_split(
    train_df['text'], train_df['label'], test_size=0.2, random_state=42, stratify=train_df['label']
)

In [15]:
# --- Load Model and Adapters ---
model_name = '/kaggle/input/deberta-v3-small/transformers/default/1'
lora_model_path = '/kaggle/input/deberta-lora-model/transformers/default/1/deberta_lora_model'

# Load the base model
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3, device_map='auto')

# Load the LoRA adapters and apply them to the base model
lora_model = PeftModel.from_pretrained(base_model, lora_model_path)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(lora_model_path)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/deberta-v3-small/transformers/default/1 and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/peft/config.py:165: UserWarning: Unexpected keyword arguments ['target_parameters'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


## 3. Probability Calibration

In [16]:
from sklearn.isotonic import IsotonicRegression
import os
os.environ["WANDB_DISABLED"] = "true"

# Create a dataset for the validation text
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt")
class TempDataset(torch.utils.data.Dataset):
    def __init__(self, encodings): self.encodings = encodings
    def __getitem__(self, idx): return {key: val[idx] for key, val in self.encodings.items()}
    def __len__(self): return len(self.encodings['input_ids'])
val_temp_dataset = TempDataset(val_encodings)

# We need a dummy trainer to run predictions easily
dummy_trainer = Trainer(model=lora_model)
val_predictions = dummy_trainer.predict(val_temp_dataset)
val_probs = torch.nn.functional.softmax(torch.from_numpy(val_predictions.predictions), dim=-1).numpy()

# Train calibrators
calibrators = {}
for i in range(3):
    iso_reg = IsotonicRegression(out_of_bounds='clip')
    y_cal = (val_labels.to_numpy() == i).astype(int)
    iso_reg.fit(val_probs[:, i], y_cal)
    calibrators[i] = iso_reg

print("Calibration models trained.")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Calibration models trained.


## 4. Generate Final Submission

In [18]:
# Create dataset for the test text
test_encodings = tokenizer(test_df['text'].tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt")
test_temp_dataset = TempDataset(test_encodings)

# Predict on the test set
test_predictions = dummy_trainer.predict(test_temp_dataset)
test_probs = torch.nn.functional.softmax(torch.from_numpy(test_predictions.predictions), dim=-1).numpy()

# Apply calibration
calibrated_probs = np.zeros_like(test_probs)
for i in range(3):
    calibrated_probs[:, i] = calibrators[i].predict(test_probs[:, i])

# Normalize probabilities
calibrated_probs_sum = np.sum(calibrated_probs, axis=1, keepdims=True)
normalized_probs = calibrated_probs / (calibrated_probs_sum + 1e-9)

# Create submission file
submission_df['winner_model_a'] = normalized_probs[:, 0]
submission_df['winner_model_b'] = normalized_probs[:, 1]
submission_df['winner_tie'] = normalized_probs[:, 2]

submission_df.to_csv('submission.csv', index=False)

submission_df.head()

,id,winner_model_a,winner_model_b,winner_tie
0,136060,0.212494,0.349897,0.437609
1,211333,0.384959,0.353800,0.261241
2,1233961,0.359485,0.342486,0.298029
